In [1]:
import pandas as pd

In [15]:
# how often does a 20 beat a dealer with a 5 when dealer hits on soft 17
import blackjack_utils.game_config as gc
import blackjack_utils.shoe as shoe
import blackjack_utils.card as card
import random

# create a shoe with 6 decks
deck = shoe.Shoe(6)
player_cards = [card.Card().from_ints(11, 0), card.Card().from_ints(9, 0)]
dealer_card_up = card.Card().from_ints(3, 0)
deck.remove(player_cards[0])
deck.remove(player_cards[1])
deck.remove(dealer_card_up)
game_config = gc.GameConfig(6, True, True, True, 1.5)
results = []
for i in range(100000):
    deck_copy = shoe.Shoe(6)
    deck_copy.cards = deck.cards.copy()
    deck_copy.shuffle()
    dealer_cards = [dealer_card_up, deck_copy.draw()]
    result = game_config.evaluate(player_cards, dealer_cards, deck_copy)
    results.append(result)


In [ ]:
mean = sum(results) / len(results)
print(mean) # 0.67191 - interpretation: expected value is + 0.67 - you profit 67% of your bet on average

0.67191


In [17]:
# now what happens when the player stays on 17 vs an 8
deck = shoe.Shoe(6)
player_cards = [card.Card().from_ints(11, 0), card.Card().from_ints(5, 0)]
dealer_card_up = card.Card().from_ints(6, 0)
deck.remove(player_cards[0])
deck.remove(player_cards[1])
deck.remove(dealer_card_up)
game_config = gc.GameConfig(6, True, True, True, 1.5)
results = []
for i in range(100000):
    deck_copy = shoe.Shoe(6)
    deck_copy.cards = deck.cards.copy()
    deck_copy.shuffle()
    dealer_cards = [dealer_card_up, deck_copy.draw()]
    result = game_config.evaluate(player_cards, dealer_cards, deck_copy)
    results.append(result)


In [ ]:
mean = sum(results) / len(results)
print(mean) # -0.38408 - interpretation: expected value is -0.38408 - you lose 38% of your bet on average

-0.38408
